In [1]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import pandas as pd
import pycomlink as pycml
import pycomlink.processing.wet_dry.mlp as mlp
import pycomlink.processing.wet_dry.cnn as cnn
from pycomlink.processing.wet_dry.nearby_wetdry import * 
from pycomlink.spatial.grid_intersection import calc_sparse_intersect_weights_for_several_cmls
from pycomlink.spatial.grid_intersection import get_grid_time_series_at_intersections

import tqdm

2024-05-30 12:33:31.005110: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# load data from your favourite sandbox path
ds_cml_all = xr.open_dataset('./data/openMRG_example.nc')
ds_rad = xr.open_dataset('./data/openMRG_example_rad.nc')

In [3]:
# calculate total loss
ds_cml_all["tl"] = ds_cml_all.tsl - ds_cml_all.rsl

In [4]:
# works best with one month of data, as this could potentially capture a rainfall event

# remove cmls with strong diurnal cycles 
ds_cml_all = ds_cml_all.where(
    ((ds_cml_all.tl.rolling(time = 60*5, center = True).std() > 2).sum(
        dim = 'time') / ds_cml_all.time.size <= 0.1).all(dim = 'sublink_id'),
    drop = True
)

# remove cmls with very noisy periods
ds_cml_all = ds_cml_all.where(
    (ds_cml_all.tl.rolling(time = 60, center = True).std() > 0.8).sum(
        dim = 'time') / ds_cml_all.time.size <= 0.35,
    drop = True
)

In [5]:
# Delay grids to allow dask to track them
intersect_weights = calc_sparse_intersect_weights_for_several_cmls(
    x1_line=ds_cml_all.site_0_lon.values,
    y1_line=ds_cml_all.site_0_lat.values,
    x2_line=ds_cml_all.site_1_lon.values,
    y2_line=ds_cml_all.site_1_lat.values,
    cml_id=ds_cml_all.cml_id.values,
    x_grid=ds_rad.lon.values,
    y_grid=ds_rad.lat.values, # 
)
    

In [6]:
start = ds_rad.time[0].values
end = ds_rad.time[-1].values

grid_data = ds_rad.sel(time=slice(start, end)).rainfall
R_rad = get_grid_time_series_at_intersections(
    grid_data = grid_data*(1/60), # to sum 1 min
    intersect_weights=intersect_weights,
).resample(time = '1min').bfill()

In [7]:
ds_cml_all['R_rad'] = (('time', 'cml_id'), np.zeros([ds_cml_all.time.size, ds_cml_all.cml_id.size]))
ds_cml_all['R_rad'].loc[{'time':R_rad.time}] = R_rad

In [8]:
# # wet period according to graf
# roll_std_dev = ds_cml_all.tl.rolling(time=60, center=True).std() 
# threshold = 1.12*roll_std_dev.quantile(0.8,dim='time')
# ds_cml_all["wet"] = roll_std_dev>threshold

# seperate periods of rain from dry time steps
# ds_cml_all["wet"] = ds_cml_all.tl.rolling(time=60, center=True).std(skipna=False) > 0.8
ds_cml_all['wet'] = (('cml_id', 'time'), np.zeros([ds_cml_all.cml_id.size, ds_cml_all.time.size]))
for cml_id in tqdm.tqdm(ds_cml_all.cml_id.values):
   mlp_out = mlp.mlp_wet_dry(
       trsl_channel_1 = ds_cml_all.sel(cml_id = cml_id).isel(sublink_id = 0).tl.values,
       trsl_channel_2 = ds_cml_all.sel(cml_id = cml_id).isel(sublink_id = 1).tl.values,
       model_sel = 'gauge'
   )
   ds_cml_all['wet'].loc[{'cml_id':cml_id}]=  np.argmax(mlp_out, axis = 1)

# ds_cml_all['wet'] = (('cml_id', 'time'), np.zeros([ds_cml_all.cml_id.size, ds_cml_all.time.size]))
# for cml_id in tqdm.tqdm(ds_cml_all.cml_id.values):
#    cnn_out = cnn.cnn_wet_dry(
#        trsl_channel_1 = ds_cml_all.sel(cml_id = cml_id).isel(sublink_id = 0).tl.values,
#        trsl_channel_2 = ds_cml_all.sel(cml_id = cml_id).isel(sublink_id = 1).tl.values,
#    )
#    ds_cml_all['wet'].loc[{'cml_id':cml_id}]=  cnn_out > 0.82

# wet when radar is wet
# radar_rain_thrs = 0.00167 # 0.001 correspods to 0.06 mm per hour # 00167 to 0.1
# ds_cml_all['wet'] = ds_cml_all['R_rad'] > radar_rain_thrs 

100%|████████████████████████████████████| 357/357 [08:02<00:00,  1.35s/it]


In [9]:
ds_cml_all = ds_cml_all.isel(sublink_id = 0)

In [10]:
# extend the wet period 
# def smooth_binary_array(arr, window_size=1):
#     # Create a kernel for smoothing, e.g., [1, 1, 1] for a simple moving average
#     kernel = np.ones(window_size, dtype=bool)
    
#     # Use convolution to smooth the array
#     smoothed_arr = np.convolve(arr, kernel, mode='same')
    
#     # Convert non-zero values to True
#     smoothed_arr = smoothed_arr > 0
    
#     return smoothed_arr
# for cml_id in tqdm.tqdm(ds_cml_all.cml_id.values):
#     ds_cml_all['wet'].loc[{'cml_id':cml_id}] = smooth_binary_array(ds_cml_all.sel(cml_id = cml_id).wet.values, window_size=30)

In [11]:
# # set baseline using median of last wet and dry events
# ds_cml_all['baseline'] = (('time', 'cml_id'), np.zeros([ds_cml_all.time.size, ds_cml_all.cml_id.size]))
# delta = 60*6 # number of surrounding dry minutes to use
# for i, cml_id in tqdm.tqdm(enumerate(ds_cml_all.cml_id)):
    
#     tl = ds_cml_all.sel(cml_id = cml_id).tl.data.copy()
#     wet = ds_cml_all.sel(cml_id = cml_id).wet.data.astype(bool).copy()
#     baseline = tl.copy() # use tl as base for baseline
    
#     for t in range(len(baseline)):
#         if wet[t]: # modify baseline array if timestep is wet
#             # Set left and right bounds for storing median value
#             left_b = t # init at where we start
#             right_b = t

#             # Iterate to find first dry timesteps
#             while wet[left_b] & (left_b > 0): # find first point that is not wet
#                 left_b -= 1
#             # while wet[right_b] & (right_b- 1 < wet.size):
#             #     right_b += 1

#             n_dry = 0 # count number of dry steps inside interval (levt_b, right_b)
#             while n_dry < delta: # increase bounds until number of dry events is equal to delta_init
#                 left_b-=2 # advance to left
#                 while wet[left_b] & (left_b > 0): # advance more if period is wet
#                     left_b -= 1
                
#                 # right_b+=1
#                 # while wet[right_b] & (right_b- 1 < wet.size):
#                 #     right_b += 1
                    
#                 if left_b < 0:
#                     break  # use current number of dry minutes
#                 # if right_b > wet.size:
#                 #     right_b = wet.size
#                 n_dry = (~wet[left_b:right_b]).sum()

#             dry_steps = ~wet[left_b:right_b]
            
#             baseline[t] = np.median(tl[left_b:right_b][dry_steps])
#     ds_cml_all['baseline'].loc[{'cml_id':cml_id}] = baseline

In [12]:
#ds_cml_all = ds_cml_all.sel(time = slice(ds_rad.time[0], ds_rad.time[-1]))

In [26]:
# estimate the baseline during rain events
ds_cml_all["baseline"] = pycml.processing.baseline.baseline_constant(
    trsl=ds_cml_all.tl,
    wet=ds_cml_all.wet,
    n_average_last_dry=5,
)


# # compenmsate for wet antenna attenuation
# cmls["waa"] = pycml.processing.wet_antenna.waa_schleiss_2013(
#     rsl=cmls.tl,
#     baseline=cmls.baseline,
#     wet=cmls.wet,
#     waa_max=2.2,
#     delta_t=1,
#     tau=15,
# )
# ds_cml_all["A_obs"] = ds_cml_all.tl - ds_cml_all.baseline
# ds_cml_all["A_obs"] = ds_cml_all.A_obs.where(ds_cml_all.A_obs >= 0, 0)

# ds_cml_all["waa"] = pycml.processing.wet_antenna.waa_leijnse_2008_from_A_obs(
#     A_obs=ds_cml_all.A_obs,
#     f_Hz=ds_cml_all.frequency * 1e6,
#     pol=ds_cml_all.polarization,
#     L_km=ds_cml_all.length / 1000,
# )
# # WAA from GRaf2020
# ds_cml_all["A_obs"] = ds_cml_all.tl - ds_cml_all.baseline
# ds_cml_all["A_obs"] = ds_cml_all.A_obs.where(ds_cml_all.A_obs >= 0, 0)
# ds_cml_all["waa"] = pycml.processing.wet_antenna.waa_leijnse_2008_from_A_obs(
#     A_obs=ds_cml_all.A_obs,
#     f_Hz=ds_cml_all.frequency*1e6,
#     pol=ds_cml_all.polarization,
#     L_km=ds_cml_all.length/ 1000, 
#     gamma = 1.47e-5, # Parameter that determines the magnitutde of the water film thickness (graf2020)
#     l_antenna = 0.0041, # antanna cover material tickness [meter] (graf2020)
#     delta = 0.36, # factor for the relation between the nonlinarity of rain rate and water film tickness
# )

ds_cml_all["A_obs"] = ds_cml_all.tl - ds_cml_all.baseline # can use different baselines
ds_cml_all["A_obs"] = ds_cml_all.A_obs.where(ds_cml_all.A_obs >= 0, 0)

ds_cml_all["waa"] = pycml.processing.wet_antenna.waa_pastorek_2021_from_A_obs(
    A_obs=ds_cml_all.A_obs,
    f_Hz=ds_cml_all.frequency * 1e6,
    pol=ds_cml_all.polarization.values,
    L_km=ds_cml_all.length/ 1000,
    A_max=9,
    zeta=0.55, # 0.55 is default
)



# calculate attenuation caused by rain and remove negative attenuation
ds_cml_all["A"] = ds_cml_all.tl - ds_cml_all.baseline - ds_cml_all.waa
ds_cml_all["A"].values[ds_cml_all.A < 0] = 0
# derive rain rate via the k-R relation
ds_cml_all["R"] = pycml.processing.k_R_relation.calc_R_from_A(
    A=ds_cml_all.A,
    L_km=ds_cml_all.length.astype(float)/1000, # convert to km
    f_GHz=ds_cml_all.frequency/1000, # convert to GHz
    pol=ds_cml_all.polarization,
)

In [27]:
# ds_cml_res = ds_cml_all.where(ds_cml_all.wet.mean(dim = 'time') > 0.05, drop = True).sel(
#     time = slice('2015-07-25T00:00', '2015-07-25T15:00')).isel(sublink_id = 0).R.resample(time = '5T').sum(skipna = True)/60

In [28]:
# slice data, resample to 5 min, convert to sum 5 min
ds_cml_res = ds_cml_all.sel(time = slice(ds_rad.time[0], ds_rad.time[-1])).resample(time = '5min').sum(skipna = True)/60

In [29]:
# store in project folder
ds_cml_res.to_netcdf('./data/ds_cml_ompenmrg.nc')